# <font color='blue'>Data Science Academy Big Data Real-Time Analytics com Python e Spark</font>

# <font color='blue'>Capítulo 9</font>

## <font color='blue'>Spark MLLib - Regressão Linear</font>

<strong> Descrição </strong>
<ul style="list-style-type:square">
  <li>Método para avaliar o relacionamento entre variáveis.</li>
  <li>Estima o valor de uma variável dependente a partir dos valores das variáveis independentes.</li>
  <li>Usado quando as variáveis dependente e independente são contínuas e possuem alguma correlação.</li>
  <li>O R-Square mede quão perto os dados estão da linha de regressão. O valor do R-Squared será entre 0 e 1, sendo que quanto maior o valor, melhor.</li>
  <li>Os dados de entrada e de saída são usados na construção do modelo. A equação linear retorna os valores dos coeficientes.</li>
  <li>A equação linear representa o modelo.</li>
</ul>

<dl>
  <dt>Vantagens</dt>
  <dd>- Baixo custo</dd>
  <dd>- Veloz</dd>
  <dd>- Excelente para realação lineares</dd>
  <br />
  <dt>Desvantagens</dt>
  <dd>- Somente variáveis numéricas</dd>
  <dd>- Sensível a outliers</dd>
  <br />
  <dt>Aplicação</dt>
  <dd>- Um dos modelos mais antigos e pode ser usado para resolver diversos problemas</dd>
</dl>

## Usaremos Regressão Linear para prever os valores de MPG (Miles Per Gallon)

MPG será a variável target e as demais variáveis serão as features (variáveis preditoras).

In [28]:
# Spark Session - usada quando se trabalha com Dataframes no Spark
spark = SparkSession.builder.getOrCreate()

In [29]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

In [33]:

# @hidden_cell
credentials_1 = {
  'auth_url':'https://identity.open.softlayer.com',
  'project':'object_storage_f0d6ce32_5e0f_4bc0_8812_229b8d429dbe',
  'project_id':'9a0cc60102244d368e96a83f25d4ca89',
  'region':'dallas',
  'user_id':'0caf8026c98a4342ac027a05416e6dee',
  'domain_id':'3be46074545f4c09b1f10df3ace95998',
  'domain_name':'1351407',
  'username':'member_327b95c3eecf105b8bdb0125b81968cfcc557dbd',
  'password':"""D[Cvr1bgf9DM^I{C""",
  'container':'CursoSpark',
  'tenantId':'undefined',
  'filename':'carros.csv'
}


In [34]:

from pyspark.sql import SparkSession

# @hidden_cell
# This function is used to setup the access of Spark to your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def set_hadoop_config_with_credentials_f0d6ce325e0f4bc08812229b8d429dbe(name):
    """This function sets the Hadoop configuration so it is possible to
    access data from Bluemix Object Storage using Spark"""

    prefix = 'fs.swift.service.' + name
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + '.auth.url', 'https://identity.open.softlayer.com'+'/v3/auth/tokens')
    hconf.set(prefix + '.auth.endpoint.prefix', 'endpoints')
    hconf.set(prefix + '.tenant', '9a0cc60102244d368e96a83f25d4ca89')
    hconf.set(prefix + '.username', '0caf8026c98a4342ac027a05416e6dee')
    hconf.set(prefix + '.password', 'D[Cvr1bgf9DM^I{C')
    hconf.setInt(prefix + '.http.port', 8080)
    hconf.set(prefix + '.region', 'dallas')
    hconf.setBoolean(prefix + '.public', False)

# you can choose any name
name = 'keystone'
set_hadoop_config_with_credentials_f0d6ce325e0f4bc08812229b8d429dbe(name)


In [36]:
# Leitura do arquivo no Hadoop
fileNameOut = 'swift://'+ credentials_1['container'] + '.keystone/carros.csv' 
carrosRDD = sc.textFile(fileNameOut)

In [37]:
# Colocando o RDD em cache. Esse processo otimiza a performance.
carrosRDD.cache()

swift://CursoSpark.keystone/carros.csv MapPartitionsRDD[49] at textFile at NativeMethodAccessorImpl.java:-2

In [38]:
carrosRDD.count()

399

In [39]:
carrosRDD.take(5)

[u'MPG,CYLINDERS,DISPLACEMENT,HORSEPOWER,WEIGHT,ACCELERATION,MODELYEAR,NAME',
 u'18,8,307,130,3504,12,70,chevrolet chevelle malibu',
 u'15,8,350,165,3693,11.5,70,buick skylark 320',
 u'18,8,318,150,3436,11,70,plymouth satellite',
 u'16,8,304,150,3433,12,70,amc rebel sst']

In [40]:
# Removendo a primeira linha do arquivo (cabeçalho)
carrosRDD2 = carrosRDD.filter(lambda x: "DISPLACEMENT" not in x)
carrosRDD2.count()

398

## Limpeza dos Dados

In [41]:
# Usando um valor padrão para average HP (que será usado para preencher os valores missing)
mediaHP = sc.broadcast(75.0)

In [42]:
# Função para limpeza dos dados
def limpaDados( inputStr) :
    global mediaHP
    attList = inputStr.split(",")
    
    # Substitui o caracter ? por um valor
    hpValue = attList[3]
    if hpValue == "?":
        hpValue = mediaHP.value
       
    # Cria uma linha usando a função Row, limpando e convertendo os dados de string para float
    linhas = Row(MPG = float(attList[0]), CYLINDERS = float(attList[1]), DISPLACEMENT = float(attList[2]), 
                 HORSEPOWER = float(hpValue), WEIGHT = float(attList[4]), ACCELERATION = float(attList[5]), 
                 MODELYEAR = float(attList[6]), NAME = attList[7]) 
    return linhas

In [44]:
# Executa a função no RDD
carrosRDD3 = carrosRDD2.map(limpaDados)
carrosRDD3.cache()
carrosRDD3.take(5)

[Row(ACCELERATION=12.0, CYLINDERS=8.0, DISPLACEMENT=307.0, HORSEPOWER=130.0, MODELYEAR=70.0, MPG=18.0, NAME=u'chevrolet chevelle malibu', WEIGHT=3504.0),
 Row(ACCELERATION=11.5, CYLINDERS=8.0, DISPLACEMENT=350.0, HORSEPOWER=165.0, MODELYEAR=70.0, MPG=15.0, NAME=u'buick skylark 320', WEIGHT=3693.0),
 Row(ACCELERATION=11.0, CYLINDERS=8.0, DISPLACEMENT=318.0, HORSEPOWER=150.0, MODELYEAR=70.0, MPG=18.0, NAME=u'plymouth satellite', WEIGHT=3436.0),
 Row(ACCELERATION=12.0, CYLINDERS=8.0, DISPLACEMENT=304.0, HORSEPOWER=150.0, MODELYEAR=70.0, MPG=16.0, NAME=u'amc rebel sst', WEIGHT=3433.0),
 Row(ACCELERATION=10.5, CYLINDERS=8.0, DISPLACEMENT=302.0, HORSEPOWER=140.0, MODELYEAR=70.0, MPG=17.0, NAME=u'ford torino', WEIGHT=3449.0)]

## Análise Exploratória de Dados

In [45]:
# Cria um Dataframe
carrosDF = spSession.createDataFrame(carrosRDD3)

In [46]:
# Estatísticas descritivas
carrosDF.select("MPG","CYLINDERS").describe().show()

+-------+-----------------+------------------+
|summary|              MPG|         CYLINDERS|
+-------+-----------------+------------------+
|  count|              398|               398|
|   mean|23.51457286432161| 5.454773869346734|
| stddev|7.815984312565782|1.7010042445332125|
|    min|              9.0|               3.0|
|    max|             46.6|               8.0|
+-------+-----------------+------------------+



In [48]:
# Encontrando a correlação entre a variável target com as variáveis preditoras
for i in carrosDF.columns:
    if not(isinstance(carrosDF.select(i).take(1)[0][0], str)) :
        print( "Correlação da variável MPG com ", i, carrosDF.stat.corr('MPG', i))

('Correla\xc3\xa7\xc3\xa3o da vari\xc3\xa1vel MPG com ', 'ACCELERATION', 0.4202889121016501)
('Correla\xc3\xa7\xc3\xa3o da vari\xc3\xa1vel MPG com ', 'CYLINDERS', -0.7753962854205548)
('Correla\xc3\xa7\xc3\xa3o da vari\xc3\xa1vel MPG com ', 'DISPLACEMENT', -0.8042028248058979)
('Correla\xc3\xa7\xc3\xa3o da vari\xc3\xa1vel MPG com ', 'HORSEPOWER', -0.7747041523498721)
('Correla\xc3\xa7\xc3\xa3o da vari\xc3\xa1vel MPG com ', 'MODELYEAR', 0.5792671330833091)
('Correla\xc3\xa7\xc3\xa3o da vari\xc3\xa1vel MPG com ', 'MPG', 1.0)


IllegalArgumentException: u'requirement failed: Currently correlation calculation for columns with dataType StringType not supported.'

## Pré-Processamento dos Dados

In [49]:
# Convertendo para um LabeledPoint (target, Vector[features])
# Remove colunas não relevantes para o modelo ou com baixa correlação
def transformaVar(row) :
    obj = (row["MPG"], Vectors.dense([row["ACCELERATION"], row["DISPLACEMENT"], row["WEIGHT"]]))
    return obj

In [50]:
# Utiliza o RDD, aplica a função, converte para Dataframe e aplica a função select()
carrosRDD4 = carrosRDD3.map(transformaVar)
carrosDF = spSession.createDataFrame(carrosRDD4,["label", "features"])
carrosDF.select("label","features").show(10)

+-----+-------------------+
|label|           features|
+-----+-------------------+
| 18.0|[12.0,307.0,3504.0]|
| 15.0|[11.5,350.0,3693.0]|
| 18.0|[11.0,318.0,3436.0]|
| 16.0|[12.0,304.0,3433.0]|
| 17.0|[10.5,302.0,3449.0]|
| 15.0|[10.0,429.0,4341.0]|
| 14.0| [9.0,454.0,4354.0]|
| 14.0| [8.5,440.0,4312.0]|
| 14.0|[10.0,455.0,4425.0]|
| 15.0| [8.5,390.0,3850.0]|
+-----+-------------------+
only showing top 10 rows



In [51]:
carrosRDD4.take(5)

[(18.0, DenseVector([12.0, 307.0, 3504.0])),
 (15.0, DenseVector([11.5, 350.0, 3693.0])),
 (18.0, DenseVector([11.0, 318.0, 3436.0])),
 (16.0, DenseVector([12.0, 304.0, 3433.0])),
 (17.0, DenseVector([10.5, 302.0, 3449.0]))]

## Machine Learning

In [52]:
# Dados de Treino e de Teste
(dados_treino, dados_teste) = carrosDF.randomSplit([0.7, 0.3])

In [53]:
dados_treino.count()

272

In [54]:
dados_teste.count()

126

In [55]:
# Construindo o modelo com os dados de treino
linearReg = LinearRegression(maxIter = 10)
modelo = linearReg.fit(dados_treino)

In [56]:
print(modelo)

LinearRegression_4491917793456fd86cfe


In [57]:
# Imprimindo as métricas
print("Coeficientes: " + str(modelo.coefficients))
print("Intercept: " + str(modelo.intercept))

Coeficientes: [0.155651372686,-0.014123376903,-0.00585712917211]
Intercept: 41.3320904343


In [58]:
# Previsões com dados de teste
predictions = modelo.transform(dados_teste)
predictions.select("prediction", "features").show()

+------------------+-------------------+
|        prediction|           features|
+------------------+-------------------+
|12.202199008001678|[18.5,304.0,4732.0]|
|16.640552331132675|[11.0,350.0,3664.0]|
| 9.849247388015257|[11.0,429.0,4633.0]|
|11.983326531447936|[12.5,350.0,4499.0]|
|15.054145224740232|[13.0,350.0,3988.0]|
| 12.12140651661755|[13.5,350.0,4502.0]|
|11.184900274614865| [8.5,440.0,4312.0]|
|10.818999259086471| [9.0,454.0,4354.0]|
|18.387367045107613|[10.0,455.0,3086.0]|
|11.326505834633554|[11.5,400.0,4464.0]|
| 14.14556409160954|[13.5,351.0,4154.0]|
| 15.14046016200571|[14.0,318.0,4077.0]|
| 15.13276489725651| [9.5,400.0,3761.0]|
|14.503575082561689|[13.0,350.0,4082.0]|
|18.798875880116704|[12.0,304.0,3433.0]|
|20.424259684825586|[15.5,225.0,3439.0]|
| 18.30151414441597|[17.0,250.0,3781.0]|
|21.403301490674348|[18.0,250.0,3278.0]|
| 19.21689074852226|[18.0,258.0,3632.0]|
|24.701146871145287|[14.5,121.0,2933.0]|
+------------------+-------------------+
only showing top

In [59]:
# Coeficiente de determinação R2
avaliador = RegressionEvaluator(predictionCol = "prediction", labelCol = "label", metricName = "r2")
avaliador.evaluate(predictions)

0.6494247503752931

# Fim

### Obrigado - Data Science Academy - <a href=http://facebook.com/dsacademy>facebook.com/dsacademybr</a>